<a href="https://colab.research.google.com/github/kim0129s/pdm15/blob/main/py-tensorflow/pdm15_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## +추가내용

### 1. transfer learning이란?
transfer learning(TL)이란 전이학습이라고 하는데 어느 특정 분야에서 학습된 신경망의 일부능력을 유사하거나 전혀 새로운 분야에서 사용되는 신경망의 학습에 
이용하는 것을 의미합니다. Resnet이나 VGG등 과 같은 신경망의 구성 중 앞단은 CNN레이어로 구성되어 있고, 이 CNN레이어는 이미지의 특징을 추출하는
능력이 있습니다. 첫번째는 신형성을 추출하고 다음에는 패턴을, 마지막에는 형상 등을 추출한다고 알려져 있는데 특징을 추출하는 신경망의 능력은 다른 분야에서 
활용될 수 있습니다. 전이 학습은 학습 데이터의 수가 적을때 효과적이고 학습 속도도 빠릅니다. 전이학습 없이 학습하는 것 보다 훨씬 높은 정확도를 제공한다는
장점이 있습니다.

### 2. cifar10 TL 결과 요약
 


In [8]:
from tensorflow.keras.applications.mobilenet_v2 import  MobileNetV2, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
print(tf.__version__)

2.7.0


In [2]:
tf.keras.backend.clear_session() 


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("#### ", len(gpus), "Physical GPUs,",
                 len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('#### No CUDA supported GPU in this computer.')

####  1 Physical GPUs, 1 Logical GPUs


In [3]:
from keras.datasets import cifar10

(trainX, trainy), (testX, testy) = cifar10.load_data()
from sklearn.model_selection import train_test_split
X_train, X_test,y_train , y_test = train_test_split(trainX,trainy,test_size=0.1,stratify=trainy, random_state=42)

X_train, y_train,X_test, y_test = X_test,y_test, testX, testy
print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))

170508288/170498071 [==============================] - 2s 0us/step
Train: X=(5000, 32, 32, 3), y=(5000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [4]:
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    )
datagen.fit(X_train)

In [5]:
img_rows,img_cols, channels = 32,32,3

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)
input_shape = (img_rows, img_cols, 1)
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (5000, 32, 32, 3)
5000 train samples
10000 test samples


In [6]:
X_train=X_train.astype("float32")  
X_test=X_test.astype("float32")

In [7]:
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [9]:
def plothist(hist):
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [10]:
TL_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
TL_base.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                

## cifar10


In [ ]:

from keras.models import load_model
model = load_model('./model/cifar10_pct10_mobilenetV2S_SGD_best_weights.148-0.8741.hdf5')